In [2]:
import os
import time
from collections import defaultdict

import openai
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import tiktoken

import utils
from utils import extract_score

label_to_score = utils.label_to_score

load_dotenv()
openai.api_key = os.getenv("OPENAI_KEY")

In [ ]:
speaker_minutes = pd.read_csv("../data/minutes.csv")

In [ ]:
prompt_template = """
<minutes>
INPUT
</minutes>
<labels>
Dovish: Strongly expresses a belief that the economy may be
growing too slowly and may need stimulus through mon-
etary policy.
Mostly dovish: Overall message expresses a belief that the economy may
be growing too slowly and may need stimulus through
monetary policy.
Neutral: Expresses neither a hawkish nor dovish view and is
mostly objective.
Mostly hawkish: Overall message expresses a belief that the economy is
growing too quickly and may need to be slowed down
through monetary policy.
Hawkish: Strongly expresses a belief that the economy is growing
too quickly and may need to be slowed down through monetary policy.
</labels>
Which label best applies applies to the minutes text as a whole above (Dovish, Mostly Dovish, Neutral, Mostly Hawkish, Hawkish)?
"""

In [ ]:
scores = defaultdict(int)

In [ ]:
for i, minutes in speaker_minutes.iterrows():
    text = minutes["minutes_text"]
    response = None
    while response is None:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4-32k",
                messages=[
                    {"role": "user", "content": prompt_template.replace("INPUT", text)},
                ],
                temperature=0,
            )
        except Exception as e:
            print(e)
            time.sleep(1)
    score = extract_score(response["choices"][0]["message"]["content"])
    print(minutes["date"], score)
    scores[minutes["date"]] = score

In [ ]:
import pandas as pd

pd.DataFrame(scores.items(), columns=["date", "score"]).to_csv(
    "../results/minutes_scores.csv", index=False
)